In [155]:
import re
import os
import ast
import mdpd
import numpy as np
import pandas as pd

In [154]:
# interceptamos las respuestas reales contra las predichas
df = pd.read_excel("ignorar/comprar_resultados.xlsx")
df['columnas realidad'] = df['columnas realidad'].apply(ast.literal_eval)
df['columnas ejecuciones'] = df['columnas ejecuciones'].apply(ast.literal_eval)

for i, fila in df.iterrows():
    if not fila['ejecutar']:
        continue
    respuesta_real = pd.DataFrame()
    
    with open(fila['archivo realidad'],'r') as f:
        contenido:str = f.read()
        inicio = contenido.find("INFO: resultado: '''") - len("INFO: resultado: '''") - 1
        fin = contenido.find(" '''\nWARNING: }") - 1
        
        respuesta_real = mdpd.from_md(contenido[inicio : fin])
    
    respuestas_predichas = []
    with open(fila['archivo ejecuciones'], 'r') as f:
        contenido:str = f.read()
        
        inicios = [inicio.end () for inicio in re.finditer("INFO: resultado: '''", contenido)]
        finales = [final.start() for final in re.finditer(" '''\nWARNING: }", contenido)]
        tiempos = [final.group(1) for final in re.finditer(r"INFO: tiempo: (\d*.\d*),", contenido)]
        for i in range(len(inicios)):
            respuestas_predichas.append(mdpd.from_md(contenido[inicios[i] + 1: finales[i]-1]))
    
    
    resultados = []
    
    for prediccion, tiempo in zip(respuestas_predichas, tiempos):
        
        resumen = {}
        resumen['Cantidad de respuestas esperadas'] = len(respuesta_real)
        resumen['Cantidad de respuesta predichas'] = len(prediccion) 
        resumen['Tiempo de ejecución'] = tiempo

        if prediccion.empty:
            prediccion = pd.DataFrame(columns=fila['columnas ejecuciones'])
        
        # Aqui procesaremos los datos de las predicciones por si tienen
        # algun caracter extraño. Caracteres extraños detectados:
        #    - \xa0: Un especie de espacio en blanco que no es el mismo caracter
        #            que el verdadero espacio en blanco
        
        tuplas_prediccion = []
        for record in prediccion.itertuples(index=False, name=None):
            tupla = []
            for i in range(len(fila['columnas ejecuciones'])):
                if isinstance(record[i], str):
                    tupla.append(record[i].replace(u'\xa0', u' '))
                else:
                    tupla.append(record[i])
                    
            tuplas_prediccion.append(tuple(tupla))
        
        tuplas_realidad = list(respuesta_real.itertuples(index=False, name=None))
        
        resumen['Cantidad de respuestas predichas que son correctas'] = len(set(tuplas_realidad).intersection(set(tuplas_prediccion)))
        resultados.append(resumen)   
    
    with open(fila['archivo resultado'], 'w') as f:
        for i, resultado in enumerate(resultados):
            f.write(f"\n@ Metricas de la ejecución {i+1}\n")
            f.write(f"metricas: {resultado}\n")

In [174]:
# Juntamos todas las metricas
carpeta_metricas = 'resultados\metricas\ln\llama2-uncensored'
nombre_archivos = os.listdir(carpeta_metricas)
for i, nombre in enumerate(nombre_archivos):
    resumenes = []
    with open(os.path.join(carpeta_metricas, nombre),'r') as f:
        contenido = f.read()
        resumenes = [ast.literal_eval(diccionario.group(1)) for diccionario in re.finditer(r'metricas: (.*)\n',contenido)]
    
    precision_avg = 0
    recall_avg = 0
    time_avg = 0
    for resumen in resumenes:
        if resumen['Cantidad de respuesta predichas'] != 0:
            precision_avg += (resumen['Cantidad de respuestas predichas que son correctas'] / resumen['Cantidad de respuesta predichas']) * 1/len(resumenes)
        
        recall_avg += (resumen['Cantidad de respuestas predichas que son correctas']/ resumen['Cantidad de respuestas esperadas'])* 1/len(resumenes)
        
        time_avg += float(resumen['Tiempo de ejecución']) * 1/len(resumenes)
    
    with open(os.path.join(carpeta_metricas, 'resumen.txt'),'a') as f:
        f.write(f"\n@ Resumen de las metricas de la ejecución {i+1}\n")
        f.write(f"Precisión media: {precision_avg}\nRecall media: {recall_avg}\ntiempo medio: {time_avg}\n")